In [2]:
import psycopg2
import pandas as pd

In [25]:
# Initiate constant
HOST = "127.0.0.1"
DBNAME = "world_accounts"
USER = "postgres"
PASSWORD = "naufalaldy23"

## Building functions

In [28]:
def create_database():
    # Connect to default db
    conn = psycopg2.connect("host={} user={} password={}".format(HOST, USER, PASSWORD))
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # Create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS {}".format(DBNAME))
    cur.execute("CREATE DATABASE {}".format(DBNAME))
    
    # Close connection with default db
    cur.close()
    
    # Connect to sparkify database
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(HOST, DBNAME, USER, PASSWORD))
    cur = conn.cursor()
    
    return cur, conn

In [5]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [8]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

## Read data from csv to store in db later

In [41]:
AccountSeries = pd.read_csv("data/wealthaccounts-series.csv")

In [42]:
AccountSeries.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [43]:
# Drop unused column
AccountSeries_clean = AccountSeries.drop(['Previous Indicator Code', 'Previous Indicator Name'], axis = 1)

In [44]:
AccountSeries_clean.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...


In [18]:
AccountsData = pd.read_csv("data/wealthaccounts-data.csv")

In [19]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,6.610924e+10,6.805651e+10,6.853525e+10,7.075229e+10,7.160031e+10,7.248767e+10,7.372787e+10,7.571698e+10,7.804556e+10,8.121498e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,2.632626e+04,2.716086e+04,2.833368e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3.072504e+03,3.178417e+03,3.312927e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,1.781710e+04,1.844594e+04,1.924519e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3.872142e+03,3.992188e+03,4.161137e+03


In [20]:
# Select only few column
AccountsData = AccountsData[['Country Code', 'Country Name', 'Series Name', 'Series Code']]

In [21]:
AccountsData.head()

,Country Code,Country Name,Series Name,Series Code
0,ALB,Albania,Human capital (constant 2018 US$),NW.HCA.TO
1,ALB,Albania,Human capital per capita (constant 2018 US$),NW.HCA.PC
2,ALB,Albania,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC
3,ALB,Albania,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC
4,ALB,Albania,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC


In [22]:
AccountsData.columns

Index(['Country Code', 'Country Name', 'Series Name', 'Series Code'], dtype='object')

## Database Action

### Connect to db

In [60]:
cur, conn = create_database()

ObjectInUse: database "world_accounts" is being accessed by other users
DETAIL:  There are 3 other sessions using the database.


### Create tables

In [32]:
# Create series table and commit 
series_table_create = ("""CREATE TABLE IF NOT EXISTS accountseries(
code VARCHAR PRIMARY KEY,
indicator_name VARCHAR,
long_definition VARCHAR,
source VARCHAR,
topic VARCHAR,
measure_unit VARCHAR,
periodicity VARCHAR,
ref_period VARCHAR,
methodology VARCHAR
)""")

In [34]:
cur.execute(series_table_create)
conn.commit()

In [61]:
# Create data table 
data_table_create = ("""CREATE TABLE IF NOT EXISTS accountdata(
country_code VARCHAR PRIMARY KEY,
country_name VARCHAR,
series_name VARCHAR,
series_code VARCHAR)""")

In [62]:
cur.execute(data_table_create)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


### Insert data

In [40]:
insert_to_series_table = ("""INSERT INTO accountseries(
code,
indicator_name,
long_definition,
source,
topic,
measure_unit,
periodicity,
ref_period,
methodology)
VALUES ("%s, %s, %s, %s, %s, %s, %s, %s, %s")
""")

In [47]:
for i, row in AccountSeries_clean.iterrows():
    print((row))

Code                                                                           NW.HCA.TO
Indicator Name                                         Human capital (constant 2018 US$)
Long definition                        Human capital is computed as the present value...
Source                                 World Bank. 2021. The Changing Wealth of Natio...
Topic                                                                      Human capital
Unit of measure                                                        Constant 2018 US$
Periodicity                                                                       Annual
Reference period                                                               1995-2018
Statistical concept and methodology    Total wealth is calculated by summing up estim...
Name: 0, dtype: object
Code                                                                           NW.HCA.PC
Indicator Name                              Human capital per capita (constant 2018 US$

In [55]:
insert_to_data_table = ("""INSERT INTO accountseries(
country_code,
country_name,
series_name,
series_code)
VALUES ("%d, %s, %s, %s")
""")

In [56]:
for i, row in AccountSeries_clean.iterrows():
    cur.execute(insert_to_data_table, list(row))
    conn.commit()

ValueError: unsupported format character 'd' (0x64) at index 91